# Taller 1 Cadenas de Markov



**Libraries**
-

In [166]:
import pandas as pd
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 

# **1. Carga y Preprocesamiento de Datos**

## 1.1 Carga de Base de Datos

In [167]:
df = pd.read_excel("Divisas_2008_2025.xlsx")

## 1.2 Análisis Exploratorio
- Este apartado busca examinar la estructura y composición de los datos, con el fin de ejecutar una serie de transformaciones que brinden estabilidad para el manejo y adaptación de los datos.

In [168]:
df

,Fecha,Euro - USD/EUR - Tasa media(Dato fin de semana),Euro - USD/EUR - Tasa media(Dato fin de semestre),Dólar australiano - COP/AUD - Tasa media(Dato fin de semana),Dólar australiano - COP/AUD - Tasa media(Dato fin de semestre),Dólar canadiense - COP/CAD - Tasa media(Dato fin de semana),Dólar canadiense - COP/CAD - Tasa media(Dato fin de semestre),Euro - COP/EUR - Tasa media(Dato fin de semana),Euro - COP/EUR - Tasa media(Dato fin de semestre)
0,20/11/2008,-,-,-,-,-,-,-,-
1,21/11/2008,-,-,-,-,-,-,-,-
2,22/11/2008,"1,25095",-,"1.457,24220",-,"1.831,66938",-,"2.946,87542",-
3,23/11/2008,"1,25095",-,"1.457,24220",-,"1.831,66938",-,"2.946,87542",-
4,24/11/2008,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...
6098,03/08/2025,"1,15545",-,"2.668,22473",-,"2.994,61184",-,"4.771,33833",-
6099,04/08/2025,-,-,-,-,-,-,-,-
6100,05/08/2025,-,-,-,-,-,-,-,-
6101,06/08/2025,-,-,-,-,-,-,-,-


**Observaciones**
- Se utiliza el "-" como señalizador de que no existe ningún valor
- Los números utilizan como separador de miles "." y para números decimales ",".
- La ausencia de valores se debe a la fecha, debido a que cada valor depende del tiempo establecido de la recolección del dato (Ver columnas).
- Las últimos 4 registros no poseen datos al no ser fechas de fin de semana o fin de semestre.

In [169]:
rows, columns = df.shape
print(f"El tamaño del Dataframe es de {rows} filas x {columns}")

El tamaño del Dataframe es de 6103 filas x 9


In [170]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6103 entries, 0 to 6102
Data columns (total 9 columns):
 #   Column                                                          Non-Null Count  Dtype 
---  ------                                                          --------------  ----- 
 0   Fecha                                                           6103 non-null   object
 1   Euro - USD/EUR - Tasa media(Dato fin de semana)                 6103 non-null   object
 2   Euro - USD/EUR - Tasa media(Dato fin de semestre)               6103 non-null   object
 3   Dólar australiano - COP/AUD - Tasa media(Dato fin de semana)    6103 non-null   object
 4   Dólar australiano - COP/AUD - Tasa media(Dato fin de semestre)  6103 non-null   object
 5   Dólar canadiense - COP/CAD - Tasa media(Dato fin de semana)     6103 non-null   object
 6   Dólar canadiense - COP/CAD - Tasa media(Dato fin de semestre)   6103 non-null   object
 7   Euro - COP/EUR - Tasa media(Dato fin de semana)              

**Observaciones**
- Las columnas son de tipo objeto (cadena de texto), lo que implica una conversión de tipo a númerico decimal.
- La columna de Fecha Requiere segmentación en día, mes y año, para un mejor manejo de los datos.

In [171]:
for column in df.columns:
    print(f"Existen {df[column].isna().sum()} en la columna {column}")

Existen 0 en la columna Fecha
Existen 0 en la columna Euro - USD/EUR - Tasa media(Dato fin de semana)
Existen 0 en la columna Euro - USD/EUR - Tasa media(Dato fin de semestre)
Existen 0 en la columna Dólar australiano - COP/AUD - Tasa media(Dato fin de semana)
Existen 0 en la columna Dólar australiano - COP/AUD - Tasa media(Dato fin de semestre)
Existen 0 en la columna Dólar canadiense - COP/CAD - Tasa media(Dato fin de semana)
Existen 0 en la columna Dólar canadiense - COP/CAD - Tasa media(Dato fin de semestre)
Existen 0 en la columna Euro - COP/EUR - Tasa media(Dato fin de semana)
Existen 0 en la columna Euro - COP/EUR - Tasa media(Dato fin de semestre)


**Observaciones**
- No haber valores faltantes (nan) diferentes a los denotados con "-".

In [172]:
df['Fecha'].unique()

array(['20/11/2008', '21/11/2008', '22/11/2008', ..., '05/08/2025',
       '06/08/2025', '07/08/2025'], dtype=object)

**Observaciones**
- Existen valores no congruentes con el formato del dataframe (ej:'Descargado de sistema del Banco de la República jueves, 7 de agosto de 2025 2:02:36 p.\xa0m.)

## 1.3 Transformación de Datos

**Acciones a Realizar**
1. Separar el campo Fecha en 3 columnas (día, mes y año)
2. Renombrar las columnas 
3. Eliminación de registros irrelevantes (no aportan información)
4. Transformación de Tipos 
5. Realizar conversiones de divisas (USD a COP).


### **1. Separar el campo Fecha en 3 columnas (día, mes y año)**

In [173]:
date_dict = {
    "Day": [],
    "Month": [],
    "Year": []
}

def divide_date(date : str):
    '''
        Extrae de cada registro de la columnna 'Fecha' 
        el día, mes y año
        
        Parámetro:
        - date: registro de 'Fecha'

        Retorna:
        - date: registro de fecha
    '''

    if pd.isna(date):
        return date
    
    if '/' not in date:
        return date

    date_parts = date.split("/")

    date_dict["Day"].append(date_parts[0])
    date_dict["Month"].append(date_parts[1])
    date_dict["Year"].append(date_parts[2])

    return date

# Se llena el diccionario con los valores seccionados
df['Fecha'].apply(divide_date)
# Se unen el diccionario al dataset
df = pd.concat([df, pd.DataFrame(date_dict)],axis=1)
# Se borra la columna Fecha
df = df.drop(columns=['Fecha'])

### **2. Renombramiento de columnas**

**Convención a Utilizar**
- Con el fin de simplificar los nombres del dataset, se agregará la palabra COP, seguido de la divisa, de la siguiente forma: COP_DIVISA
- Debido a que se tienen distintas columnas en base a datos temporales, se establecerá una convención de la siguiente forma:
  - Semana -> W (ej: COP_EURO_W)
  - Semestre -> S (ej: COP_USD_S)
- Dado que los dólares estadounidenses no se encontraron directamente convertidos a pesos colombianos, se denotará de forma similar pero con USD, aunque posteriormente se hará su respectiva conversión a pesos colombianos.

In [174]:
new_name_dict = {
    "Euro - USD/EUR - Tasa media(Dato fin de semana)": "USD_EURO_W",
    "Euro - USD/EUR - Tasa media(Dato fin de semestre)": "USD_EURO_S",
    "Dólar australiano - COP/AUD - Tasa media(Dato fin de semana)":"COP_AUD_W",
    "Dólar australiano - COP/AUD - Tasa media(Dato fin de semestre)":"COP_AUD_S",
    "Dólar canadiense - COP/CAD - Tasa media(Dato fin de semana)":"COP_CAD_W",
    "Dólar canadiense - COP/CAD - Tasa media(Dato fin de semestre)":"COP_CAD_S",
    "Euro - COP/EUR - Tasa media(Dato fin de semana)":"COP_EUR_W",
    "Euro - COP/EUR - Tasa media(Dato fin de semestre)":"COP_EUR_S"
}
# Se pasa el diccionario, donde la clave es el nombre antiguo y el valor el nuevo nombre
df = df.rename(new_name_dict,axis=1)
df.head()

,USD_EURO_W,USD_EURO_S,COP_AUD_W,COP_AUD_S,COP_CAD_W,COP_CAD_S,COP_EUR_W,COP_EUR_S,Day,Month,Year
0,-,-,-,-,-,-,-,-,20,11,2008
1,-,-,-,-,-,-,-,-,21,11,2008
2,"1,25095",-,"1.457,24220",-,"1.831,66938",-,"2.946,87542",-,22,11,2008
3,"1,25095",-,"1.457,24220",-,"1.831,66938",-,"2.946,87542",-,23,11,2008
4,-,-,-,-,-,-,-,-,24,11,2008


### **3. Eliminación de registros irrelevantes (no aportan información)**
- Dada la existencia de registros donde no existen valores para los precios (al no ser fechas de fin de semana o semestre), se procede a eliminarlos del dataset.

In [175]:
# Extracción de registros con información
df = df[(df['USD_EURO_W'] != '-') | (df['USD_EURO_S'] != '-')]
# Reset del índice
df = df.reset_index(drop=True)

df

,USD_EURO_W,USD_EURO_S,COP_AUD_W,COP_AUD_S,COP_CAD_W,COP_CAD_S,COP_EUR_W,COP_EUR_S,Day,Month,Year
0,"1,25095",-,"1.457,24220",-,"1.831,66938",-,"2.946,87542",-,22,11,2008
1,"1,25095",-,"1.457,24220",-,"1.831,66938",-,"2.946,87542",-,23,11,2008
2,"1,26890",-,"1.509,48160",-,"1.867,54753",-,"2.941,31020",-,29,11,2008
3,"1,26890",-,"1.509,48160",-,"1.867,54753",-,"2.941,31020",-,30,11,2008
4,"1,26770",-,"1.483,66473",-,"1.794,96173",-,"2.958,22865",-,06,12,2008
...,...,...,...,...,...,...,...,...,...,...,...
1761,"1,16475",-,"2.610,19799",-,"2.916,21532",-,"4.659,71049",-,20,07,2025
1762,"1,17305",-,"2.696,26935",-,"2.999,58404",-,"4.821,79856",-,26,07,2025
1763,"1,17305",-,"2.696,26935",-,"2.999,58404",-,"4.821,79856",-,27,07,2025
1764,"1,15545",-,"2.668,22473",-,"2.994,61184",-,"4.771,33833",-,02,08,2025


### **4. Transformación de Tipos**
- Se procede a quitar el separador de miles "." y colocar el indicador de decimales "." en vez de ","
- En los campos donde no existe ningun valor, se colocará un valor por defecto de NaN perteneciente a NumPy (np.nan).
- Se transformará la columna a númerico decimal (float) o entero, dependiendo si es una columna de carácter temporal o monetaria.

In [176]:
integer_columns = ['Day','Month','Year']

for column in df.columns:
    # Remplazo de separadores
    df[column] = df[column].replace("-",np.nan,regex=False)
    df[column] = df[column].str.replace(".","",regex=False)
    df[column] = df[column].str.replace(",",".",regex=False)
    
    # Conversión a tipos dependiendo del tipo de columna (Temporal o Monetaria)
    if column not in integer_columns:
        df[column] = df[column].astype(float)
    else:
        df[column] = df[column].astype(int)

df.head()

,USD_EURO_W,USD_EURO_S,COP_AUD_W,COP_AUD_S,COP_CAD_W,COP_CAD_S,COP_EUR_W,COP_EUR_S,Day,Month,Year
0,1.25095,NaN,1457.24220,NaN,1831.66938,NaN,2946.87542,NaN,22,11,2008
1,1.25095,NaN,1457.24220,NaN,1831.66938,NaN,2946.87542,NaN,23,11,2008
2,1.26890,NaN,1509.48160,NaN,1867.54753,NaN,2941.31020,NaN,29,11,2008
3,1.26890,NaN,1509.48160,NaN,1867.54753,NaN,2941.31020,NaN,30,11,2008
4,1.26770,NaN,1483.66473,NaN,1794.96173,NaN,2958.22865,NaN,6,12,2008
